In [22]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [23]:
# gerais

import pandas as pd
from tqdm import tqdm


# Gensim
from nltk import tokenize
import gensim.corpora as corpora
from gensim.models import CoherenceModel

#plot
import matplotlib
matplotlib.use("TkAgg")  # Define o backend ANTES de importar pyplot

import matplotlib.pyplot as plt  # Só importar depois de definir o backend

#import pyLDAvis
#import pyLDAvis.gensim_models



import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from unidecode import unidecode


from string import punctuation

from gensim.models import LdaMulticore, CoherenceModel


In [24]:
df = pd.read_csv('../datasets/fakeWhatsApp.BR_2022.csv')

In [25]:
len(df)

598971

In [26]:
minimo = df['date_message'].min()
maximo = df['date_message'].max()

print(maximo)
print(minimo)

2022-11-14 09:13:45
2022-09-27 00:00:08


In [27]:
df.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5
0,2022-10-10 18:20:24,9d737b3c9387855139bbad2311cc5709,0638569ee76dac58f59dcac20463c955,NaN,NaN,NaN,False,False,False,☝️\nHoje que eu fique sabendo do CANAL LULA FL...,5,2022-10-10 18:20:25.000937,-0.7003,0.843775,F7023FFB06C429A2C166922849A35ED8,558594228826.0:12@s.whatsapp.net,Texto,whatsapp,NaN,NaN
1,2022-10-10 22:02:58,1660a60f661754d2802ca53296e25be8,a5910d5cc1c830ade9eb4dd00f15ff6a,NaN,NaN,NaN,False,False,False,Mais pra que isso não aconteça nois temos quê ...,5,2022-10-10 22:02:58.4682,-0.2960,NaN,A9FAC78070C144890D181EF415B90CAD,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
2,2022-10-11 00:39:31,c882172d447798d74915973ac83eba68,b84dfe2d1599b82768dcdecce7e6bb23,d2e0ec59ffd9f84764f5b147725d7196.oga,audio/ogg; codecs=opus,NaN,True,False,False,NaN,5,2022-10-11 00:39:33.445125,NaN,NaN,737948BE86D450A426470794F91BC80D,558594228826.0:12@s.whatsapp.net,Audio,whatsapp,NaN,d2e0ec59ffd9f84764f5b147725d7196
3,2022-10-10 23:36:19,91e3c22c08b24ba01ac4524d77bcb1da,addb88a34374d43aa9ecd4df7359ce39,NaN,NaN,NaN,False,False,False,‎Acesse este link para entrar no meu grupo do ...,5,2022-10-10 23:36:19.724987,-0.1531,NaN,439A91ADD8F355CD23C4BB107A5E88BB,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
4,2022-10-10 23:40:12,77c1a8a31fee269db258a028a61f0b88,a5831b109d4d95fae8ee0ce464e48a6a,bb80cd530332bb6b95c34e719992d249.f4v,video/mp4,NaN,True,False,False,NaN,5,2022-10-10 23:40:14.397495,NaN,NaN,092203A082AC3DFB2A4933F60453AEB8,558594228826.0:12@s.whatsapp.net,Video,whatsapp,NaN,bb80cd530332bb6b95c34e719992d249


In [28]:
df.columns

Index(['date_message', 'id_member_anonymous', 'id_group_anonymous', 'media',
       'media_type', 'media_url', 'has_media', 'has_media_url', 'trava_zap',
       'text_content_anonymous', 'dataset_info_id', 'date_system',
       'score_sentiment', 'score_misinformation', 'id_message', 'id_persona',
       'message_type', 'messenger', 'media_name', 'media_md5'],
      dtype='object')

In [29]:
ids_para_remover = ['6ef561ec0f448afcd7b3751124bb0712','d3e678a0ba0e1485548260a7c4599152',
                    'ac5703154484de05336af617455ca55e','a39edbd64d378226ffa60433649a0acf',
                    'c66d0d4ae5a4b281bff67e1fa4fbd6ba','819bbc872ed6d81f44d746b710eecf06',
                    'ec94da4d54f9a5693e88fa582926be53','6a38c72316d87c028dfd66c10442476b',
                    '94099e1e46f129856541e2b3640896d1','8f367d1693fff47218603fa47ded525c',
                    'c09caffee0d1bd30926dea9df25dc88f']

In [30]:
# Filtrar o DataFrame para remover as linhas com esses IDs
df_filtrado = df[~df['id_member_anonymous'].isin(ids_para_remover)]
df_filtrado

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5
0,2022-10-10 18:20:24,9d737b3c9387855139bbad2311cc5709,0638569ee76dac58f59dcac20463c955,NaN,NaN,NaN,False,False,False,☝️\nHoje que eu fique sabendo do CANAL LULA FL...,5,2022-10-10 18:20:25.000937,-0.7003,0.843775,F7023FFB06C429A2C166922849A35ED8,558594228826.0:12@s.whatsapp.net,Texto,whatsapp,NaN,NaN
1,2022-10-10 22:02:58,1660a60f661754d2802ca53296e25be8,a5910d5cc1c830ade9eb4dd00f15ff6a,NaN,NaN,NaN,False,False,False,Mais pra que isso não aconteça nois temos quê ...,5,2022-10-10 22:02:58.4682,-0.2960,NaN,A9FAC78070C144890D181EF415B90CAD,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
2,2022-10-11 00:39:31,c882172d447798d74915973ac83eba68,b84dfe2d1599b82768dcdecce7e6bb23,d2e0ec59ffd9f84764f5b147725d7196.oga,audio/ogg; codecs=opus,NaN,True,False,False,NaN,5,2022-10-11 00:39:33.445125,NaN,NaN,737948BE86D450A426470794F91BC80D,558594228826.0:12@s.whatsapp.net,Audio,whatsapp,NaN,d2e0ec59ffd9f84764f5b147725d7196
3,2022-10-10 23:36:19,91e3c22c08b24ba01ac4524d77bcb1da,addb88a34374d43aa9ecd4df7359ce39,NaN,NaN,NaN,False,False,False,‎Acesse este link para entrar no meu grupo do ...,5,2022-10-10 23:36:19.724987,-0.1531,NaN,439A91ADD8F355CD23C4BB107A5E88BB,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
4,2022-10-10 23:40:12,77c1a8a31fee269db258a028a61f0b88,a5831b109d4d95fae8ee0ce464e48a6a,bb80cd530332bb6b95c34e719992d249.f4v,video/mp4,NaN,True,False,False,NaN,5,2022-10-10 23:40:14.397495,NaN,NaN,092203A082AC3DFB2A4933F60453AEB8,558594228826.0:12@s.whatsapp.net,Video,whatsapp,NaN,bb80cd530332bb6b95c34e719992d249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598966,2022-11-13 00:46:56,b1b419f54664f6cd4fcdb40425862d36,b1b419f54664f6cd4fcdb40425862d36,7719ba57cbe74eb5c8059d6b1a14d1ca.f4v,video/mp4,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.262572,NaN,NaN,08E682716448AB06DEE126A30E7AAA77,558594228826.0:17@s.whatsapp.net,Video,whatsapp,NaN,7719ba57cbe74eb5c8059d6b1a14d1ca
598967,2022-11-13 13:37:03,2ad743e9ff831a75911a983b42205845,64c9a7c8ac338faa258e5aea9a1c2ae3,c6d59f0b86e63dbcce7fc682a1eaea50.f4v,video/mp4,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.269632,NaN,NaN,3EB0259ED9B8FACF18A9,558594228826.0:17@s.whatsapp.net,Video,whatsapp,NaN,c6d59f0b86e63dbcce7fc682a1eaea50
598968,2022-11-13 17:43:22,b1b419f54664f6cd4fcdb40425862d36,b1b419f54664f6cd4fcdb40425862d36,003ace942d9665adc9647df9d9526c0c.jpeg,image/jpeg,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.277283,NaN,NaN,5CD9F23CD07BEE38693E68E11F9C6A55,558594228826.0:17@s.whatsapp.net,Imagem,whatsapp,NaN,003ace942d9665adc9647df9d9526c0c
598969,2022-11-13 21:40:47,b1b419f54664f6cd4fcdb40425862d36,b1b419f54664f6cd4fcdb40425862d36,bab5653a94b097404739a061dbbc7407.f4v,video/mp4,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.284021,NaN,NaN,D3BDE4A5C68E7148C0925F6C582C99A9,558594228826.0:17@s.whatsapp.net,Video,whatsapp,NaN,bab5653a94b097404739a061dbbc7407


In [31]:
# Removendo linhas com valores NaN na coluna 'text_content_anonymous'
df_filtrado = df_filtrado.dropna(subset=['text_content_anonymous'])

# Removendo trava_zap
df_filtrado = df_filtrado[df_filtrado['trava_zap'] == False]



#df_filtrado = df_filtrado[0:100]
len(df_filtrado)

254480

In [32]:
token_espaco = tokenize.WhitespaceTokenizer()
token_pontuacao = tokenize.WordPunctTokenizer()

In [34]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('portuguese'))

custom_stopwords = {
    't', 'be', 'nao', 'youtu', 'vai', 'av', 'ja', 'to', 'the', 'this', 'i', 'and', 
    'you', 'y', 'www', 'sao', 'pois', 'contra', 'user', 'ai', 'so', 'gente', 'voce', 'of', 
    'ola', 'gift', 'card', 'kwaivideo', 'r', 'bom', 'q', 'vc', 'vcs', 'pra','ta', 'phone', 'ok', 'la',
    'sera', 'ha', 'aqui', 'ate', 'dia', 'mc', 'im', 'tmj', 'pix', 'g', 'diz', 'ti', 'etc', 'tudo', 
    'todo', 'toda', 'youtube', 'g1', 'm', 'instagram', 'fb', 'in', 'link', 'was', 'blocked', '  '
}

stop_words.update(custom_stopwords)

def preprocess_text(text):

    # Função para extrair e substituir o domínio da URL
    def substituir_dominios(texto):
        # Função para extrair e substituir o domínio da URL
        def extrair_dominio(url):
            # Remove o protocolo (http://, https://, etc.) e o "www." se presente
            dominio = re.sub(r'^https?://(?:www\.)?|www\.', '', url)
            # Remove o caminho e parâmetros da URL
            dominio = re.split(r'[/?#]', dominio)[0]
            # Retorna a parte principal do domínio (antes do primeiro ponto)
            return dominio.split('.')[0]

        # Substitui URLs por seus domínios principais
        return re.sub(r'https?://(?:www\.)?\S+|www\.\S+', lambda match: extrair_dominio(match.group(0)), texto)

    # Substituir domínios
    text = substituir_dominios(text)

    # Converte para minúsculas
    text = text.lower()

    # Remove acentos
    text = unidecode(text)

    #Remover Pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove URLs e menções
    #text = re.sub(r'http\S+|www\S+|https\S+|@\w+', '', text)

    # Substitui emojis repetidos por apenas um
    text = re.sub(r'([\U00010000-\U0010FFFF])\1+', r'\1', text)
    text = re.sub(r'([\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F700-\U0001F77F]|[\U0001F780-\U0001F7FF]|[\U0001F800-\U0001F8FF]|[\U0001F900-\U0001F9FF]|[\U0001FA00-\U0001FA6F]|[\U0001FA70-\U0001FAFF])\1+', r'\1', text)


    # Remove espaços em branco extras (início ou final) e múltiplos espaços no meio do texto
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove pontuações e caracteres especiais
    #text = re.sub(r'[^\w\s]', '', text)

    # Ajusta risadas "kkk" ou mais para "kk"
    text = re.sub(r'k{2,}|K{2,}', 'kk', text)

    # Ajusta risadas "haha" ou mais para "haha"
    text = re.sub(r'(ha){2,}', 'haha', text, flags=re.IGNORECASE)

    # Ajusta risadas "kaka" ou mais para "kaka"
    text = re.sub(r'(ka){2,}', 'kaka', text, flags=re.IGNORECASE)

    # Remove as stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])


    return text


[nltk_data] Downloading package punkt to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [35]:

df_geral = df_filtrado.copy()

# Aplicar o pré-processamento à coluna de texto
df_geral['text_processed'] = df_geral['text_content_anonymous'].apply(preprocess_text)

In [36]:
df_geral.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,...,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5,text_processed
0,2022-10-10 18:20:24,9d737b3c9387855139bbad2311cc5709,0638569ee76dac58f59dcac20463c955,NaN,NaN,NaN,False,False,False,☝️\nHoje que eu fique sabendo do CANAL LULA FL...,...,2022-10-10 18:20:25.000937,-0.7003,0.843775,F7023FFB06C429A2C166922849A35ED8,558594228826.0:12@s.whatsapp.net,Texto,whatsapp,NaN,NaN,hoje fique sabendo canal lula flix vi pt entro...
1,2022-10-10 22:02:58,1660a60f661754d2802ca53296e25be8,a5910d5cc1c830ade9eb4dd00f15ff6a,NaN,NaN,NaN,False,False,False,Mais pra que isso não aconteça nois temos quê ...,...,2022-10-10 22:02:58.4682,-0.2960,NaN,A9FAC78070C144890D181EF415B90CAD,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,aconteca nois fazer anossa parte
3,2022-10-10 23:36:19,91e3c22c08b24ba01ac4524d77bcb1da,addb88a34374d43aa9ecd4df7359ce39,NaN,NaN,NaN,False,False,False,‎Acesse este link para entrar no meu grupo do ...,...,2022-10-10 23:36:19.724987,-0.1531,NaN,439A91ADD8F355CD23C4BB107A5E88BB,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,acesse entrar grupo whatsapp chat
7,2022-10-10 16:49:48,325720ed3339a91b8076df12c1e95c45,0e345813dcb62b0fe4d8537f311af0f1,NaN,NaN,https://m.kwai.com/photo/150000006567403/52240...,False,True,False,Fortes palavras da Pastora Damares!\nhttps://k...,...,2022-10-10 16:49:50.051126,0.0000,NaN,F18BADED5AFA8FB0C33FE36625872DB8,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,fortes palavras pastora damares
8,2022-10-10 16:49:53,8ed44a70a011285622e2b8919c2c8c3e,3b3e64e81c3e3f3bfc5ba76ddb90fb2d,0d6f2896e5941ef1933e69bbd7a32f69.jpeg,image/jpeg,NaN,True,False,False,🤣🤣🤣🤣🤣🤣🤣🤣,...,2022-10-10 16:49:54.713831,0.0000,NaN,724C233591C3A62A5231B47FF22ADE4B,558594228826.0:12@s.whatsapp.net,Imagem,whatsapp,NaN,0d6f2896e5941ef1933e69bbd7a32f69,


### Calculando a diversidade dos tópicos
A diversidade de tópicos mede quantas palavras únicas existem entre os top-N termos de todos os tópicos. 

In [37]:

def topic_diversity(model, top_n=10):
    topic_words = []
    for k in range(model.num_topics):
        words = [word for word, _ in model.show_topic(k, topn=top_n)]
        topic_words.extend(words)
    unique_words = set(topic_words)
    return len(unique_words) / (top_n * model.num_topics)


### IRBO 

IRBO avalia quanto os tópicos são distintos entre si, usando o Rank-Biased Overlap (RBO) invertido. Quanto maior o IRBO, mais diferentes são os tópicos

In [38]:
import rbo

def compute_irbo(model, top_n=10):
    topics = []
    for k in range(model.num_topics):
        topic_terms = [word for word, _ in model.show_topic(k, topn=top_n)]
        topics.append(topic_terms)

    # Calcular média dos RBOs invertidos entre todos os pares
    n = len(topics)
    total_irbo = 0
    count = 0

    for i in range(n):
        for j in range(i + 1, n):
            rbo_score = rbo.RankingSimilarity(topics[i], topics[j]).rbo()
            total_irbo += (1 - rbo_score)
            count += 1

    return total_irbo / count if count > 0 else 0


In [39]:
def processar_texto(texto):
    texto = texto.lower()
    palavras_texto = token_espaco.tokenize(texto)
    palavras_texto = token_pontuacao.tokenize(' '.join(palavras_texto))
    return [palavra for palavra in palavras_texto]

texto_processado = df_geral['text_processed'].apply(processar_texto)

In [40]:
# Criar dicionário
id2word = corpora.Dictionary(texto_processado)
# Criar o Corpus: A Frequência das palavras nos documentos
corpus = [id2word.doc2bow(text) for text in texto_processado]

## Modelando os Tópicos sem filtro

In [41]:

# Função para determinar a melhor quantidade de tópicos para a modelagem
def compute_coherence_values(dictionary, corpus, texts, coherence, limit=30, start=2, step=5):
    coherence_values = []
    model_list = []
    topic_range = list(range(start, limit, step))

    for num_topics in tqdm(topic_range, desc=f"Calculando coerência ({coherence})"):
        model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=5, workers=4)
        model_list.append(model)

        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence=coherence)
        coherence_values.append(coherencemodel.get_coherence())

    optimal_index = coherence_values.index(max(coherence_values))
    optimal_num_topics = topic_range[optimal_index]
    return optimal_num_topics, model_list, coherence_values

# Calcular os valores de coerência para diferentes métricas
def calculate_coherence_for_metrics(dictionary, corpus, texts, metrics, start=2, limit=30, step=2):
    coherence_results = {}
    for metric in metrics:
        num_topics, model_list, coherence_values = compute_coherence_values(
            dictionary=dictionary,
            corpus=corpus,
            texts=texts,
            coherence=metric,
            start=start,
            limit=limit,
            step=step
        )
        coherence_results[metric] = {
            'num_topics': num_topics,
            'model_list': model_list,
            'coherence_values': coherence_values
        }
    return coherence_results

metrics = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

# Calcular os valores de coerência 
coherence_results = calculate_coherence_for_metrics(
    dictionary=id2word,
    corpus=corpus,
    texts=texto_processado,
    metrics=metrics,
    start=2,
    limit=30,
    step=2
)

# Plotar os resultados de coerência para cada métrica
for metric in metrics:
    x = list(range(2, 30, 2))
    coherence_values = coherence_results[metric]['coherence_values']
    plt.plot(x, coherence_values, marker='o', label=metric.upper())

plt.xlabel("Número de Tópicos")
plt.ylabel("Score de Coerência")
plt.legend(loc='best')
plt.title("Coerência por Métrica")
plt.grid(True)
plt.tight_layout()
plt.show()

# Mostrar os resultados de coerência para cada métrica
for metric in metrics:
    print(f"\nResultados de Coerência para {metric.upper()}:")
    for m, cv in zip(range(2, 30, 2), coherence_results[metric]['coherence_values']):
        print(f"Qtd. tópicos = {m} -> Coerência = {round(cv, 4)}")
    print(f"Melhor número de tópicos segundo {metric.upper()}: {coherence_results[metric]['num_topics']}")


Calculando coerência (c_v):   0%|          | 0/14 [00:00<?, ?it/s]

Calculando coerência (c_npmi): 100%|██████████| 14/14 [20:42<00:00, 88.78s/it] 



Resultados de Coerência para C_V:
Qtd. tópicos = 2 -> Coerência = 0.5309
Qtd. tópicos = 4 -> Coerência = 0.5356
Qtd. tópicos = 6 -> Coerência = 0.497
Qtd. tópicos = 8 -> Coerência = 0.5098
Qtd. tópicos = 10 -> Coerência = 0.4765
Qtd. tópicos = 12 -> Coerência = 0.5034
Qtd. tópicos = 14 -> Coerência = 0.4802
Qtd. tópicos = 16 -> Coerência = 0.4848
Qtd. tópicos = 18 -> Coerência = 0.4687
Qtd. tópicos = 20 -> Coerência = 0.4698
Qtd. tópicos = 22 -> Coerência = 0.4592
Qtd. tópicos = 24 -> Coerência = 0.4577
Qtd. tópicos = 26 -> Coerência = 0.4847
Qtd. tópicos = 28 -> Coerência = 0.4941
Melhor número de tópicos segundo C_V: 4

Resultados de Coerência para U_MASS:
Qtd. tópicos = 2 -> Coerência = -5.6815
Qtd. tópicos = 4 -> Coerência = -4.2266
Qtd. tópicos = 6 -> Coerência = -3.9821
Qtd. tópicos = 8 -> Coerência = -5.214
Qtd. tópicos = 10 -> Coerência = -5.0441
Qtd. tópicos = 12 -> Coerência = -4.559
Qtd. tópicos = 14 -> Coerência = -4.5549
Qtd. tópicos = 16 -> Coerência = -3.9512
Qtd. tópic

In [42]:
# tamanho do corpus
len(corpus)


254480

In [43]:

pyLDAvis.enable_notebook()  


# Escolher a métrica para visualizar
chosen_metric = 'u_mass'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.038114  0.093502       1        1  9.782468
16    -0.208777 -0.071910       2        1  9.016360
15    -0.171366 -0.123367       3        1  6.861362
18    -0.206424 -0.072348       4        1  6.590731
19    -0.054534  0.232400       5        1  6.459736
5     -0.148357  0.043340       6        1  6.172137
9     -0.147869 -0.010363       7        1  6.100716
10    -0.044483  0.197221       8        1  5.723932
17    -0.080874  0.047386       9        1  4.782781
1     -0.147339 -0.086733      10        1  4.375858
21     0.168747 -0.070324      11        1  4.023900
0      0.212707  0.027340      12        1  3.763581
12     0.157519  0.126030      13        1  3.736529
7      0.062567  0.011529      14        1  3.038895
8      0.071318  0.044690      15        1  3.034794
20     0.228381 -0.123769      16        1  2.874430
6     -0.117242 -0.136263      17        1  2.571475
14     0.046723  0.193227      18        1  2.456707
3      0.216450 -0.153013      19        1  2.326911
4      0.116084 -0.038238      20        1  2.283798
2      0.052986 -0.135543      21        1  2.035046
11     0.031897  0.005207      22        1  1.987855, topic_info=           Term          Freq         Total Category  logprob  loglift
3965         55  67971.000000  67971.000000  Default  30.0000  30.0000
3567         13  61332.000000  61332.000000  Default  29.0000  29.0000
8          lula  61582.000000  61582.000000  Default  28.0000  28.0000
22364      robo  23142.000000  23142.000000  Default  27.0000  27.0000
2047         20  23304.000000  23304.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
3799    roberto    467.082393   3102.117735  Topic22  -5.4837   2.0248
496    nordeste    463.822031   3760.269907  Topic22  -5.4907   1.8254
10       porque    503.669970  12274.529383  Topic22  -5.4083   0.7248
334     pessoas    490.803572  12892.435953  Topic22  -5.4342   0.6498
541     oficial    428.852853   2244.283859  Topic22  -5.5691   2.2631

[1601 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
8188      1  0.100556     0
8188      6  0.002073     0
8188      8  0.850062     0
8188     10  0.002073     0
8188     11  0.042503     0
...     ...       ...   ...
4744     19  0.989430   zoe
5076      1  0.047223  zona
5076     11  0.000705  zona
5076     15  0.877847  zona
5076     16  0.073653  zona

[5713 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 17, 16, 19, 20, 6, 10, 11, 18, 2, 22, 1, 13, 8, 9, 21, 7, 15, 4, 5, 3, 12])

In [44]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus))

Diversidade de Tópicos: 0.7955
IRBO médio: 0.967

Perplexidade:  -10.806419651354263


In [45]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppSemFiltro_u_mass.csv', index=False, encoding='utf-8')


In [46]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_v'  # ou 'u_mass', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.161759  0.125156       1        1  41.013751
3      0.186882  0.060925       2        1  30.829335
0     -0.021212 -0.302266       3        1  20.201144
2     -0.327429  0.116185       4        1   7.955769, topic_info=                 Term          Freq         Total Category  logprob  loglift
75378  99999999999999  32431.000000  32431.000000  Default  30.0000  30.0000
3965               55  44129.000000  44129.000000  Default  29.0000  29.0000
21372        22222222  16757.000000  16757.000000  Default  28.0000  28.0000
8                lula  47580.000000  47580.000000  Default  27.0000  27.0000
3567               13  44163.000000  44163.000000  Default  26.0000  26.0000
...               ...           ...           ...      ...      ...      ...
261         bolsonaro   3075.003222  50910.818027   Topic4  -4.9860  -0.2755
2303          urgente   1793.250721   7954.717379   Topic4  -5.5253   1.0415
218              caso   1573.089559   5495.280243   Topic4  -5.6563   1.2804
886             globo   1479.719866   5161.576827   Topic4  -5.7175   1.2819
145           governo   1338.878620   7854.969913   Topic4  -5.8175   0.7620

[327 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
99553      4  0.998132   000news
38         1  0.156981         1
38         2  0.173765         1
38         3  0.629845         1
38         4  0.039412         1
...      ...       ...       ...
25         2  0.226640  whatsapp
25         3  0.729293  whatsapp
25         4  0.007950  whatsapp
99556      4  0.999181      wion
9751       1  0.999081    xandao

[625 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3])

In [47]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus))

Diversidade de Tópicos: 0.85
IRBO médio: 0.9296

Perplexidade:  -8.557275019807767


In [48]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppSemFiltro_c_v.csv', index=False, encoding='utf-8')


In [49]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_uci'  # ou 'c_v', 'u_mass', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.242025 -0.093697       1        1  10.030154
12     0.182033 -0.026325       2        1   9.340820
19     0.139916  0.065645       3        1   7.492021
21     0.090794  0.070793       4        1   6.864907
9     -0.022246  0.225315       5        1   6.718351
10     0.167935  0.001709       6        1   5.619048
7      0.106087  0.063163       7        1   5.335418
1      0.133551  0.025122       8        1   5.196653
2      0.091042  0.052774       9        1   4.900629
17    -0.147369  0.153997      10        1   4.584180
14    -0.206917  0.001811      11        1   4.058422
0      0.118367 -0.162915      12        1   3.994758
3     -0.126663  0.208199      13        1   3.638944
16    -0.071896  0.114495      14        1   3.371857
6     -0.014394  0.012858      15        1   3.246392
5     -0.019042 -0.048806      16        1   3.049876
8     -0.080844 -0.006976      17        1   2.761231
13     0.052409 -0.118833      18        1   2.662533
4     -0.073754 -0.089560      19        1   2.619021
18    -0.076573 -0.193475      20        1   1.756580
20    -0.228408 -0.115481      21        1   1.747049
15    -0.256054 -0.139814      22        1   1.011156, topic_info=           Term          Freq         Total Category  logprob  loglift
3567         13  59005.000000  59005.000000  Default  30.0000  30.0000
3965         55  48699.000000  48699.000000  Default  29.0000  29.0000
8          lula  59147.000000  59147.000000  Default  28.0000  28.0000
261   bolsonaro  57783.000000  57783.000000  Default  27.0000  27.0000
2047         20  20210.000000  20210.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
9535        amo    357.957508    454.622323  Topic22  -5.0739   4.3550
6081    gabriel    607.930307   1165.205325  Topic22  -4.5442   3.9435
2008     homens    761.277683   1767.680878  Topic22  -4.3193   3.7516
5545   monteiro    344.182840    517.942281  Topic22  -5.1131   4.1854
904       legal    267.183670    771.532588  Topic22  -5.3663   3.5336

[1569 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
27190     11  0.994658  0000
7044       4  0.006760    04
7044       8  0.035302    04
7044       9  0.001502    04
7044      10  0.840488    04
...      ...       ...   ...
5076      10  0.019976  zona
5076      11  0.006064  zona
5076      12  0.004637  zona
5076      14  0.001427  zona
5076      19  0.240422  zona

[5411 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 13, 20, 22, 10, 11, 8, 2, 3, 18, 15, 1, 4, 17, 7, 6, 9, 14, 5, 19, 21, 16])

In [50]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus))

Diversidade de Tópicos: 0.7864
IRBO médio: 0.9679

Perplexidade:  -10.812364860255004


In [51]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppSemFiltro_c_uci.csv', index=False, encoding='utf-8')

In [52]:

pyLDAvis.enable_notebook() 

# Escolher a métrica para visualizar
chosen_metric = 'c_npmi'  # ou 'c_v', 'c_uci', 'u_mass'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis  

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.204878 -0.012697       1        1  11.657909
7      0.188959 -0.004352       2        1  11.351206
3      0.076973  0.077850       3        1   9.421692
12     0.155939 -0.083980       4        1   7.755607
17     0.120082 -0.017532       5        1   7.443729
14     0.125717 -0.111129       6        1   6.198605
13     0.021722  0.135094       7        1   5.667582
5      0.119344 -0.017135       8        1   5.392409
10     0.090904  0.122690       9        1   4.666535
2      0.002507 -0.091167      10        1   4.314357
8     -0.143366  0.257813      11        1   4.114962
4     -0.168972 -0.027326      12        1   3.965461
6      0.088772  0.035679      13        1   3.755845
11    -0.056873 -0.218243      14        1   3.604172
0     -0.110455  0.196495      15        1   3.134473
15    -0.250173 -0.101212      16        1   2.591774
1     -0.211364 -0.183260      17        1   2.579101
16    -0.254593  0.042411      18        1   2.384580, topic_info=           Term          Freq         Total Category  logprob  loglift
3567         13  75188.000000  75188.000000  Default  30.0000  30.0000
3965         55  66208.000000  66208.000000  Default  29.0000  29.0000
261   bolsonaro  60800.000000  60800.000000  Default  28.0000  28.0000
8          lula  55274.000000  55274.000000  Default  27.0000  27.0000
2047         20  23295.000000  23295.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
479       costa    538.513811   2228.319450  Topic18  -5.5234   2.3160
170     atencao    626.883904   8054.822617  Topic18  -5.3715   1.1829
1083      apoio    568.560866   5783.682446  Topic18  -5.4691   1.4165
394        deus    572.530390  15714.366487  Topic18  -5.4621   0.4239
8          lula    546.661586  55274.087598  Topic18  -5.5084  -0.8801

[1315 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
8188       3  0.125873     0
8188       9  0.001222     0
8188      10  0.871335     0
8188      14  0.001222     0
27190      6  0.991424  0000
...      ...       ...   ...
1262       8  0.040789  zema
1262      14  0.938152  zema
1262      17  0.011654  zema
5076       3  0.007486  zona
5076      12  0.991959  zona

[4406 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 8, 4, 13, 18, 15, 14, 6, 11, 3, 9, 5, 7, 12, 1, 16, 2, 17])

In [53]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus))

Diversidade de Tópicos: 0.7833
IRBO médio: 0.9584

Perplexidade:  -10.402738716249612


In [54]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppSemFiltro_c_npmi.csv', index=False, encoding='utf-8')


In [55]:
df_geral.shape

(254480, 21)

# Modelando os Tópicos com filtro religioso

In [56]:
palavras_religiosas = [
    "deus", "jesus", "misericordia", "davi",
    "salomao", "reino", "templo", "conservador",
    "pentecostal", "rcc", "renovacao", "carismatic",
    "paulo ricardo", "bernardo kuster", "herege", "ateu",
    "jerico", "heresia"
]


In [57]:
# Função para verificar se uma palavra está relacionada à religião
def relacionada_religiao(word):
    word_lower = word.lower()
    palavras_religiosas_lower = [palavra.lower() for palavra in palavras_religiosas]

    # Verificando se alguma palavra da lista de palavras religiosas está presente
    return any(palavra in word_lower for palavra in palavras_religiosas_lower)


df_religiao = df_geral[df_geral['text_processed'].apply(lambda x: relacionada_religiao(x))]



In [58]:
len(df_religiao)

15507

In [59]:
def processar_texto(texto):
    texto = texto.lower()
    palavras_texto = token_espaco.tokenize(texto)
    palavras_texto = token_pontuacao.tokenize(' '.join(palavras_texto))
    return [palavra for palavra in palavras_texto ]

texto_processado_religiao = df_religiao['text_processed'].apply(processar_texto)

In [60]:
# Criar dicionário
id2word_religiao = corpora.Dictionary(texto_processado_religiao)
# Criar o Corpus: A Frequência das palavras nos documentos
corpus_religiao = [id2word_religiao.doc2bow(text) for text in texto_processado_religiao]

In [61]:
# tamanho do corpus
len(corpus_religiao)

15507

In [62]:

# Função para determinar a melhor quantidade de tópicos para a modelagem
def compute_coherence_values(dictionary, corpus_religiao, texts, coherence, limit=30, start=2, step=5):
    coherence_values = []
    model_list = []
    topic_range = list(range(start, limit, step))

    for num_topics in tqdm(topic_range, desc=f"Calculando coerência ({coherence})"):
        model = LdaMulticore(corpus=corpus_religiao, id2word=dictionary, num_topics=num_topics, passes=5, workers=4)
        model_list.append(model)

        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence=coherence)
        coherence_values.append(coherencemodel.get_coherence())

    optimal_index = coherence_values.index(max(coherence_values))
    optimal_num_topics = topic_range[optimal_index]
    return optimal_num_topics, model_list, coherence_values

# Calcular os valores de coerência para diferentes métricas
def calculate_coherence_for_metrics(dictionary, corpus_religiao, texts, metrics, start=2, limit=30, step=2):
    coherence_results = {}
    for metric in metrics:
        num_topics, model_list, coherence_values = compute_coherence_values(
            dictionary=dictionary,
            corpus_religiao=corpus_religiao,
            texts=texts,
            coherence=metric,
            start=start,
            limit=limit,
            step=step
        )
        coherence_results[metric] = {
            'num_topics': num_topics,
            'model_list': model_list,
            'coherence_values': coherence_values
        }
    return coherence_results

metrics = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

# Calcular os valores de coerência 
coherence_results = calculate_coherence_for_metrics(
    dictionary=id2word_religiao,
    corpus_religiao=corpus_religiao,
    texts=texto_processado_religiao,
    metrics=metrics,
    start=2,
    limit=30,
    step=2
)

# Plotar os resultados de coerência para cada métrica
for metric in metrics:
    x = list(range(2, 30, 2))
    coherence_values = coherence_results[metric]['coherence_values']
    plt.plot(x, coherence_values, marker='o', label=metric.upper())

plt.xlabel("Número de Tópicos")
plt.ylabel("Score de Coerência")
plt.legend(loc='best')
plt.title("Coerência por Métrica")
plt.grid(True)
plt.tight_layout()
plt.show()

# Mostrar os resultados de coerência para cada métrica
for metric in metrics:
    print(f"\nResultados de Coerência para {metric.upper()}:")
    for m, cv in zip(range(2, 30, 2), coherence_results[metric]['coherence_values']):
        print(f"Qtd. tópicos = {m} -> Coerência = {round(cv, 4)}")
    print(f"Melhor número de tópicos segundo {metric.upper()}: {coherence_results[metric]['num_topics']}")


Calculando coerência (c_npmi): 100%|██████████| 14/14 [11:43<00:00, 50.28s/it]


Resultados de Coerência para C_V:
Qtd. tópicos = 2 -> Coerência = 0.3501
Qtd. tópicos = 4 -> Coerência = 0.4602
Qtd. tópicos = 6 -> Coerência = 0.395
Qtd. tópicos = 8 -> Coerência = 0.4892
Qtd. tópicos = 10 -> Coerência = 0.5212
Qtd. tópicos = 12 -> Coerência = 0.4632
Qtd. tópicos = 14 -> Coerência = 0.552
Qtd. tópicos = 16 -> Coerência = 0.4859
Qtd. tópicos = 18 -> Coerência = 0.4997
Qtd. tópicos = 20 -> Coerência = 0.4959
Qtd. tópicos = 22 -> Coerência = 0.4875
Qtd. tópicos = 24 -> Coerência = 0.5159
Qtd. tópicos = 26 -> Coerência = 0.5131
Qtd. tópicos = 28 -> Coerência = 0.5173
Melhor número de tópicos segundo C_V: 14

Resultados de Coerência para U_MASS:
Qtd. tópicos = 2 -> Coerência = -2.5139
Qtd. tópicos = 4 -> Coerência = -2.6098
Qtd. tópicos = 6 -> Coerência = -2.5555
Qtd. tópicos = 8 -> Coerência = -2.5485
Qtd. tópicos = 10 -> Coerência = -2.458
Qtd. tópicos = 12 -> Coerência = -2.0849
Qtd. tópicos = 14 -> Coerência = -1.8224
Qtd. tópicos = 16 -> Coerência = -1.8226
Qtd. tópi

In [63]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'u_mass'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_religiao, id2word_religiao)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.078893  0.071574       1        1  12.312926
3      0.089234  0.041277       2        1  11.387272
9      0.088261  0.002814       3        1   8.301961
10     0.073670 -0.055467       4        1   7.978954
4      0.047030 -0.101698       5        1   7.890942
6      0.032322 -0.254692       6        1   7.166545
1      0.090371  0.056135       7        1   6.918494
5      0.080439  0.130290       8        1   6.801085
11    -0.046015  0.108131       9        1   6.088394
8      0.039447  0.088736      10        1   5.973405
2     -0.391261  0.012323      11        1   5.250281
13     0.044852 -0.076275      12        1   5.142671
7      0.038234 -0.014679      13        1   4.440603
12    -0.265477 -0.008470      14        1   4.346467, topic_info=       Term          Freq         Total Category  logprob  loglift
1267     55  27936.000000  27936.000000  Default  30.0000  30.0000
784      20   8206.000000   8206.000000  Default  29.0000  29.0000
111       1   5032.000000   5032.000000  Default  28.0000  28.0000
358    lula  10912.000000  10912.000000  Default  27.0000  27.0000
524      91   2663.000000   2663.000000  Default  26.0000  26.0000
...     ...           ...           ...      ...      ...      ...
128   apoio    274.003073   1188.808172  Topic14  -5.5727   1.6682
1817     11    258.497797    943.716413  Topic14  -5.6309   1.8409
358    lula    270.700391  10912.666866  Topic14  -5.5848  -0.5609
1242     24    252.387880    919.518883  Topic14  -5.6549   1.8429
900   acima    255.499505   3247.813853  Topic14  -5.6426   0.5933

[1242 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
3040      1  0.059450    04
3040      2  0.019109    04
3040      3  0.002123    04
3040      4  0.106161    04
3040      5  0.006370    04
...     ...       ...   ...
1457      5  0.007158  zica
1457      9  0.923398  zica
1457     11  0.007158  zica
1457     12  0.007158  zica
1457     13  0.007158  zica

[7819 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 10, 11, 5, 7, 2, 6, 12, 9, 3, 14, 8, 13])

In [64]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_religiao))

Diversidade de Tópicos: 0.5286
IRBO médio: 0.8004

Perplexidade:  -7.915327599602129


In [65]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroReligioso_u_mass.csv', index=False, encoding='utf-8')


In [66]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_v'  # ou 'u_mass', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_religiao, id2word_religiao)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.038906 -0.153981       1        1  15.248317
5     -0.077495  0.032463       2        1  14.346322
6     -0.068170  0.008959       3        1  10.471917
13    -0.058013 -0.163416       4        1  10.138482
10    -0.083830  0.054642       5        1   9.356388
4     -0.071490 -0.016954       6        1   5.963847
2     -0.072225 -0.046802       7        1   5.777743
11    -0.081985  0.063233       8        1   5.173842
3      0.427487 -0.098661       9        1   4.927383
12    -0.059219 -0.001760      10        1   4.524136
8      0.271643  0.109281      11        1   4.199528
1     -0.072221  0.008182      12        1   3.774640
0     -0.042676 -0.003655      13        1   3.083867
7      0.027101  0.208468      14        1   3.013588, topic_info=             Term          Freq         Total Category  logprob  loglift
1267           55  28174.000000  28174.000000  Default  30.0000  30.0000
784            20   8230.000000   8230.000000  Default  29.0000  29.0000
358          lula  10251.000000  10251.000000  Default  28.0000  28.0000
111             1   5039.000000   5039.000000  Default  27.0000  27.0000
32           deus  16995.000000  16995.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
970   brasileiros    269.668918   1733.051749  Topic14  -5.2224   1.6416
492         vamos    313.478148   4571.218264  Topic14  -5.0719   0.8222
304     bolsonaro    359.455354  11205.371588  Topic14  -4.9350   0.0625
645    ministerio    205.791094    682.856779  Topic14  -5.4927   2.3026
32           deus    298.772404  16995.591414  Topic14  -5.1199  -0.5390

[1232 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
48468      2  0.002787  0162
48468      4  0.002787  0162
48468      6  0.002787  0162
48468      9  0.978250  0162
48468     11  0.002787  0162
...      ...       ...   ...
2591      13  0.465511    ze
2591      14  0.004310    ze
46647     11  0.967059   zen
46647     13  0.007615   zen
46647     14  0.015229   zen

[8033 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 6, 7, 14, 11, 5, 3, 12, 4, 13, 9, 2, 1, 8])

In [67]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_religiao))

Diversidade de Tópicos: 0.5714
IRBO médio: 0.8313

Perplexidade:  -7.893997571559382


In [68]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroReligioso_c_v.csv', index=False, encoding='utf-8')


In [69]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_uci'  # ou 'c_v', 'u_mass', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_religiao, id2word_religiao)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13     0.065008 -0.086707       1        1  9.167502
9      0.029416  0.200835       2        1  9.090330
0      0.053207 -0.029221       3        1  8.982986
1     -0.489155 -0.030302       4        1  8.289730
2      0.033472 -0.151126       5        1  8.069591
11    -0.015003  0.070472       6        1  7.664650
4     -0.000528 -0.016265       7        1  7.389036
7      0.060050 -0.100631       8        1  6.926215
14     0.065501 -0.091309       9        1  5.981658
6      0.022938  0.192640      10        1  5.862634
3      0.058113 -0.037155      11        1  5.231409
12     0.029066  0.147603      12        1  4.916840
8      0.022866  0.014795      13        1  4.899427
15     0.048098 -0.063068      14        1  3.734124
10     0.053710 -0.000203      15        1  2.617697
5     -0.036761 -0.020358      16        1  1.176171, topic_info=           Term          Freq         Total Category  logprob  loglift
1267         55  29751.000000  29751.000000  Default  30.0000  30.0000
784          20   8632.000000   8632.000000  Default  29.0000  29.0000
32         deus  16852.000000  16852.000000  Default  28.0000  28.0000
274      senhor   4518.000000   4518.000000  Default  27.0000  27.0000
2435         pl   2509.000000   2509.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
22        assim    130.464970   2873.524034  Topic16  -5.0076   1.3507
304   bolsonaro    220.718568  11260.289010  Topic16  -4.4818   0.5108
2        brasil    234.174740  14347.550109  Topic16  -4.4227   0.3276
277       todos    174.103162   8529.640305  Topic16  -4.7191   0.5513
1267         55    139.474589  29751.488164  Topic16  -4.9408  -0.9198

[1381 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
48468      1  0.002652   0162
48468      2  0.002652   0162
48468      3  0.002652   0162
48468      4  0.962856   0162
48468      6  0.002652   0162
...      ...       ...    ...
1457       9  0.007247   zica
1457      11  0.007247   zica
1457      14  0.007247   zica
1457      15  0.007247   zica
21112     12  0.886486  zumba

[9374 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 10, 1, 2, 3, 12, 5, 8, 15, 7, 4, 13, 9, 16, 11, 6])

In [70]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_religiao))

Diversidade de Tópicos: 0.55
IRBO médio: 0.7918

Perplexidade:  -7.870247033879627


In [71]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroReligioso_c_uci.csv', index=False, encoding='utf-8')


In [72]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_npmi'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_religiao, id2word_religiao)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15    -0.048414  0.189072       1        1  9.873300
2      0.006741  0.114430       2        1  6.875608
6      0.006039 -0.144702       3        1  6.765491
16    -0.074333 -0.024048       4        1  6.640833
11    -0.063144 -0.022213       5        1  5.988266
5     -0.071232 -0.113953       6        1  5.978250
1     -0.052523 -0.086298       7        1  5.444141
14    -0.038910  0.188437       8        1  4.899457
10     0.465262  0.025665       9        1  4.833876
20    -0.011695 -0.070369      10        1  4.543790
3     -0.028078 -0.033641      11        1  4.011471
0      0.017431  0.100755      12        1  3.962381
8     -0.056231  0.136174      13        1  3.850596
18     0.328888 -0.053585      14        1  3.725664
4     -0.066955 -0.081512      15        1  3.492409
17    -0.048258 -0.061120      16        1  3.403697
9     -0.039901  0.049091      17        1  3.325811
13    -0.045959 -0.042036      18        1  3.311190
12    -0.075015  0.031571      19        1  3.055260
7     -0.053064 -0.079928      20        1  2.805616
19    -0.030337  0.013106      21        1  1.671565
21    -0.020312 -0.034895      22        1  1.541329, topic_info=        Term          Freq         Total Category  logprob  loglift
1267      55  28674.000000  28674.000000  Default  30.0000  30.0000
784       20   8554.000000   8554.000000  Default  29.0000  29.0000
358     lula  10319.000000  10319.000000  Default  28.0000  28.0000
32      deus  16051.000000  16051.000000  Default  27.0000  27.0000
111        1   5194.000000   5194.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
288   frente     40.817943    991.153009  Topic22  -6.4400   0.9828
235    homem     41.579062   1330.733664  Topic22  -6.4215   0.7066
401     anos     44.943961   4300.782761  Topic22  -6.3437  -0.3886
900    acima     44.069659   3367.043143  Topic22  -6.3633  -0.1635
53      pode     43.827202   3357.758816  Topic22  -6.3689  -0.1662

[1835 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
3037      1  0.038149    01
3037      2  0.027745    01
3037      3  0.003468    01
3037      4  0.603446    01
3037      5  0.003468    01
...     ...       ...   ...
1457      4  0.007526  zica
1457      7  0.007526  zica
1457      8  0.007526  zica
1457     18  0.007526  zica
1457     22  0.007526  zica

[13941 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 3, 7, 17, 12, 6, 2, 15, 11, 21, 4, 1, 9, 19, 5, 18, 10, 14, 13, 8, 20, 22])

In [73]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_religiao))

Diversidade de Tópicos: 0.5455
IRBO médio: 0.8239

Perplexidade:  -7.8237240678463555


In [74]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroReligioso_c_npmi.csv', index=False, encoding='utf-8')


## Modelando os tópicos com filtro de inclusão religioso e exclusão de termas políticos

In [75]:
palavras_politicas = [ "lula", "bolsonaro", "pt", "pl", "stf", "patria", "55", "22", "13", "senadores", "lulaladrao",
                       "urnas", "alexandre", "moraes", "comunismo", "eleicao", "eleicoes", "esquerda", "direita",
                         "presidente", "tse", "fraude", "voto", "turno", "ministro"]

In [76]:
def retirar_mensagens_com_palavras_politicas(word):
    word_lower = word.lower()
    palavras_politica_lower = [palavra.lower() for palavra in palavras_politicas]

    return any(palavra in word_lower for palavra in palavras_politica_lower)


df_politico = df_religiao[~df_religiao['text_processed'].apply(lambda x: retirar_mensagens_com_palavras_politicas(x))]



In [77]:
df_politico.shape

(5167, 21)

In [78]:
df_politico.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,...,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5,text_processed
165,2022-10-10 18:18:11,f9fd00ad20b8d4a40ad9fd2d7c64cb40,e110071613239754d38878f7e046e95b,NaN,NaN,NaN,False,False,False,Deus perdoa porque eles Não sabem que fazem e ...,...,2022-10-10 18:18:11.689928,0.4404,NaN,44C0D9A25FA1F439172A6F67EC88B3ED,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,deus perdoa porque sabem fazem falam pronto as...
215,2022-10-10 16:57:12,3d4e77cea17b1812993b55543e81400e,d01bfa0f0d31fa1307e7aeb9d3c3870e,NaN,NaN,https://www.portalcidade.news/imprensa-tendenc...,False,True,False,Imprensa tendenciosa age novamente e ataca pre...,...,2022-10-10 16:57:12.981007,0.0000,NaN,0993FD7ABC0CA5A21AED8DA493993B73,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,imprensa tendenciosa age novamente ataca prefe...
218,2022-10-10 18:34:17,71d2b615339a7c53dcf2427491da5d36,e110071613239754d38878f7e046e95b,NaN,NaN,NaN,False,False,False,Abre o esgoto que vc chama de boca porra e adm...,...,2022-10-10 18:34:17.860259,-0.7351,NaN,712BCD08CC4F7EA5514AC4A7BD6C8B9B,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,abre esgoto chama boca porra admita deus matou...
239,2022-10-10 18:35:14,f9fd00ad20b8d4a40ad9fd2d7c64cb40,e110071613239754d38878f7e046e95b,NaN,NaN,NaN,False,False,False,Deus é vida. Morreu na cruz pra dar vida. Ou s...,...,2022-10-10 18:35:14.825606,-0.3612,0.267817,9D8849865FB72F726E9B182937590C9C,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,deus vida morreu cruz dar vida acredita logo d...
333,2022-10-10 18:37:01,0842f1acc471ab358a596e0c34006c21,e110071613239754d38878f7e046e95b,NaN,NaN,NaN,False,False,False,A gente pode manda em Deus ??? Ser ele tem um ...,...,2022-10-10 18:37:01.781209,0.5514,0.116997,EABCED2ADAF7A65F48B7CD567F28534D,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,pode manda deus decreto tal cidade pode muda m...


In [79]:
def processar_texto(texto):
    texto = texto.lower()
    palavras_texto = token_espaco.tokenize(texto)
    palavras_texto = token_pontuacao.tokenize(' '.join(palavras_texto))
    return [palavra for palavra in palavras_texto ]

texto_processado_politico = df_politico['text_processed'].apply(processar_texto)

In [80]:
# Criar dicionário
id2word_politico = corpora.Dictionary(texto_processado_politico)
# Criar o Corpus: A Frequência das palavras nos documentos
corpus_politico = [id2word_politico.doc2bow(text) for text in texto_processado_politico]

In [81]:

# Função para determinar a melhor quantidade de tópicos para a modelagem
def compute_coherence_values(dictionary, corpus_politico, texts, coherence, limit=30, start=2, step=5):
    coherence_values = []
    model_list = []
    topic_range = list(range(start, limit, step))

    for num_topics in tqdm(topic_range, desc=f"Calculando coerência ({coherence})"):
        model = LdaMulticore(corpus=corpus_politico, id2word=dictionary, num_topics=num_topics, passes=5, workers=4)
        model_list.append(model)

        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence=coherence)
        coherence_values.append(coherencemodel.get_coherence())

    optimal_index = coherence_values.index(max(coherence_values))
    optimal_num_topics = topic_range[optimal_index]
    return optimal_num_topics, model_list, coherence_values

# Calcular os valores de coerência para diferentes métricas
def calculate_coherence_for_metrics(dictionary, corpus_politico, texts, metrics, start=2, limit=30, step=2):
    coherence_results = {}
    for metric in metrics:
        num_topics, model_list, coherence_values = compute_coherence_values(
            dictionary=dictionary,
            corpus_politico=corpus_politico,
            texts=texts,
            coherence=metric,
            start=start,
            limit=limit,
            step=step
        )
        coherence_results[metric] = {
            'num_topics': num_topics,
            'model_list': model_list,
            'coherence_values': coherence_values
        }
    return coherence_results

metrics = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

# Calcular os valores de coerência 
coherence_results = calculate_coherence_for_metrics(
    dictionary=id2word_politico,
    corpus_politico=corpus_politico,
    texts=texto_processado_politico,
    metrics=metrics,
    start=2,
    limit=30,
    step=2
)

# Plotar os resultados de coerência para cada métrica
for metric in metrics:
    x = list(range(2, 30, 2))
    coherence_values = coherence_results[metric]['coherence_values']
    plt.plot(x, coherence_values, marker='o', label=metric.upper())

plt.xlabel("Número de Tópicos")
plt.ylabel("Score de Coerência")
plt.legend(loc='best')
plt.title("Coerência por Métrica")
plt.grid(True)
plt.tight_layout()
plt.show()

# Mostrar os resultados de coerência para cada métrica
for metric in metrics:
    print(f"\nResultados de Coerência para {metric.upper()}:")
    for m, cv in zip(range(2, 30, 2), coherence_results[metric]['coherence_values']):
        print(f"Qtd. tópicos = {m} -> Coerência = {round(cv, 4)}")
    print(f"Melhor número de tópicos segundo {metric.upper()}: {coherence_results[metric]['num_topics']}")


Calculando coerência (c_npmi): 100%|██████████| 14/14 [03:27<00:00, 14.81s/it]


Resultados de Coerência para C_V:
Qtd. tópicos = 2 -> Coerência = 0.4712
Qtd. tópicos = 4 -> Coerência = 0.4791
Qtd. tópicos = 6 -> Coerência = 0.5303
Qtd. tópicos = 8 -> Coerência = 0.4538
Qtd. tópicos = 10 -> Coerência = 0.4285
Qtd. tópicos = 12 -> Coerência = 0.4244
Qtd. tópicos = 14 -> Coerência = 0.4417
Qtd. tópicos = 16 -> Coerência = 0.443
Qtd. tópicos = 18 -> Coerência = 0.4717
Qtd. tópicos = 20 -> Coerência = 0.4556
Qtd. tópicos = 22 -> Coerência = 0.4455
Qtd. tópicos = 24 -> Coerência = 0.4281
Qtd. tópicos = 26 -> Coerência = 0.436
Qtd. tópicos = 28 -> Coerência = 0.4494
Melhor número de tópicos segundo C_V: 6

Resultados de Coerência para U_MASS:
Qtd. tópicos = 2 -> Coerência = -2.4679
Qtd. tópicos = 4 -> Coerência = -3.4338
Qtd. tópicos = 6 -> Coerência = -3.8877
Qtd. tópicos = 8 -> Coerência = -4.3638
Qtd. tópicos = 10 -> Coerência = -4.187
Qtd. tópicos = 12 -> Coerência = -3.7233
Qtd. tópicos = 14 -> Coerência = -3.6695
Qtd. tópicos = 16 -> Coerência = -4.3725
Qtd. tópic

In [82]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'u_mass'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_politico, id2word_politico)
vis  



PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.051313  0.0       1        1  51.607944
1     -0.051313  0.0       2        1  48.392056, topic_info=         Term         Freq        Total Category  logprob  loglift
809      amem   351.000000   351.000000  Default  30.0000  30.0000
13150   nunes    99.000000    99.000000  Default  29.0000  29.0000
99     senhor  1174.000000  1174.000000  Default  28.0000  28.0000
1654    ifood    94.000000    94.000000  Default  27.0000  27.0000
1375     vivo   119.000000   119.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
269     favor    92.971271   136.637160   Topic2  -6.2939   0.3408
388      hoje   107.887549   188.749362   Topic2  -6.1451   0.1665
214     sobre   112.704925   239.429758   Topic2  -6.1014  -0.0277
435     todas    99.084700   186.394924   Topic2  -6.2302   0.0939
29      assim    98.604099   214.964453   Topic2  -6.2351  -0.0535

[194 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
5262       1  0.954647     1os
5262       2  0.056156     1os
13145      1  0.020019     673
13145      2  0.960890     673
1268       1  0.064490      96
...      ...       ...     ...
42         2  0.586582    vida
1375       1  0.083361    vivo
1375       2  0.916972    vivo
1451       1  0.916049  voltar
1451       2  0.083277  voltar

[300 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [83]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_politico))

Diversidade de Tópicos: 0.65
IRBO médio: 0.2911

Perplexidade:  -8.145927148728003


In [84]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroPolitico_u_mass.csv', index=False, encoding='utf-8')


In [85]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_v'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_politico, id2word_politico)
vis  



PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.027435 -0.022311       1        1  28.271491
0      0.053795  0.101946       2        1  17.579789
4      0.033276 -0.071471       3        1  16.261755
1      0.018980 -0.036391       4        1  14.935735
2      0.039619  0.014494       5        1  14.026905
3     -0.173105  0.013734       6        1   8.924325, topic_info=               Term         Freq        Total Category  logprob  loglift
241           jesus  1232.000000  1232.000000  Default  30.0000  30.0000
99           senhor  1175.000000  1175.000000  Default  29.0000  29.0000
13150         nunes    91.000000    91.000000  Default  28.0000  28.0000
1654          ifood    93.000000    93.000000  Default  27.0000  27.0000
9030   proprietaria    52.000000    52.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
241           jesus    31.100818  1232.988335   Topic6  -5.6984  -1.2636
110             faz    23.468697   129.664950   Topic6  -5.9800   0.7071
848           acima    24.615908   290.015708   Topic6  -5.9322  -0.0502
42             vida    24.576559   468.255750   Topic6  -5.9338  -0.5308
1270          agora    23.655079   211.135422   Topic6  -5.9720   0.2275

[487 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
875       2  0.826981      0500
224       1  0.163911       100
224       3  0.786774       100
224       6  0.032782       100
294       1  0.038731        12
...     ...       ...       ...
104       6  0.029799     voces
1451      1  0.043720    voltar
1451      2  0.072866    voltar
1451      5  0.874395    voltar
7258      1  0.901096  vomitara

[1185 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 5, 2, 3, 4])

In [86]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_politico))

Diversidade de Tópicos: 0.4833
IRBO médio: 0.4143

Perplexidade:  -8.205224103104037


In [87]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroPolitico_c_v.csv', index=False, encoding='utf-8')

In [88]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_uci'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_politico, id2word_politico)
vis  



PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.055142  0.0       1        1  60.714647
0     -0.055142  0.0       2        1  39.285353, topic_info=         Term         Freq        Total Category  logprob  loglift
848     acima   274.000000   274.000000  Default  30.0000  30.0000
1654    ifood    90.000000    90.000000  Default  29.0000  29.0000
18      video   148.000000   148.000000  Default  28.0000  28.0000
99     senhor  1191.000000  1191.000000  Default  27.0000  27.0000
247      nome   534.000000   534.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
1031  abencoe    83.435615   243.365622   Topic2  -6.1936  -0.1362
41    verdade    81.078740   207.798346   Topic2  -6.2223  -0.0068
174    cristo    87.002162   418.141316   Topic2  -6.1518  -0.6356
477      nada    80.034319   200.692952   Topic2  -6.2352   0.0150
130     fazer    80.297215   281.909826   Topic2  -6.2320  -0.3215

[191 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
224        1  0.100088      100
224        2  0.900794      100
3598       1  0.074537       15
3598       2  0.919292       15
12724      1  0.064944       64
...      ...       ...      ...
1375       2  0.064682     vivo
1451       1  0.930027   voltar
1451       2  0.069405   voltar
1468       1  0.879158  vontade
1468       2  0.127878  vontade

[294 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [89]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_politico))

Diversidade de Tópicos: 0.7
IRBO médio: 0.3881

Perplexidade:  -8.135771257933978


In [90]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroPolitico_c_uci.csv', index=False, encoding='utf-8')


In [91]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_npmi'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_politico, id2word_politico)
vis  



PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.053813  0.0       1        1  51.932189
1     -0.053813  0.0       2        1  48.067811, topic_info=        Term         Freq        Total Category  logprob  loglift
99    senhor  1183.000000  1183.000000  Default  30.0000  30.0000
220    vamos   655.000000   655.000000  Default  29.0000  29.0000
809     amem   357.000000   357.000000  Default  28.0000  28.0000
247     nome   529.000000   529.000000  Default  27.0000  27.0000
241    jesus  1238.000000  1238.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
216   tambem   126.006915   246.258548   Topic2  -5.9831   0.0625
98    sempre   130.538810   268.233489   Topic2  -5.9478   0.0124
1270   agora   112.181630   211.159733   Topic2  -6.0993   0.1001
99    senhor   174.963045  1183.711115   Topic2  -5.6549  -1.1793
29     assim   101.470272   214.794330   Topic2  -6.1997  -0.0174

[183 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
3598       1  0.048768       15
3598       2  0.950982       15
1644       1  0.035817       50
1644       2  0.967051       50
12724      1  0.063681       64
...      ...       ...      ...
18         2  0.278023    video
499        1  0.637888  vitoria
499        2  0.359834  vitoria
1451       1  0.141350   voltar
1451       2  0.848100   voltar

[276 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [92]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_politico))

Diversidade de Tópicos: 0.75
IRBO médio: 0.4583

Perplexidade:  -8.139276494429854


In [ ]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroPolitico_c_npmi.csv', index=False, encoding='utf-8')


: 